encoding utf-8 

@File       : .py
@Contact    : username.you@email.address.com
@License    : (C)Copyright AIQ Solutions 2022

@ModifyTime     @Author     @Version        @Description
-----------     -------     --------        ------------
  username    1.0             None

Module Documentation: 

In [24]:
import pydicom
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs
import dicom2nifti
import nibabel as nib
import nrrd
import glob
import os
import numpy as np
from collections import OrderedDict
import shutil


In [86]:
# import warnings
# warnings.filterwarnings("ignore")

In [25]:
root_study_dir = 'C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST'
# output_root = 'D:\\Prostate02_Test_Retest\\nii_test_retest'

In [88]:
# # remove space in folder name
# scan_dir = os.listdir(root_study_dir)
# scan_dir.sort()
# i = 0
# for scan in scan_dir:
#     os.rename(os.path.join(root_study_dir,scan),os.path.join(root_study_dir,scan.replace(" ", "")))

In [29]:
# remove RWV & nii folders
scan_dir = os.listdir(root_study_dir)
scan_dir.sort()
# i = 0
for scan in scan_dir:
    if "." not in scan:
        print("Now processing subject: " + str(scan))

        scan_session = os.listdir(os.path.join(root_study_dir,scan))
        scan_session.sort()
        for scan_modal in scan_session:
            # print("Now processing scan: " + str(scan_modal))
            scan_type = os.listdir(os.path.join(root_study_dir,scan,scan_modal))
            scan_type.sort() 
            for scan_each in scan_type:
                # print("New processing modal: ", scan_each)
                dicom_files = os.listdir(os.path.join(root_study_dir,scan,scan_modal,scan_each))
                dicom_files.sort()
                for D_files in dicom_files:
                    if (D_files == "secondary") and ("_Rt" not in scan_modal):
                        shutil.rmtree(os.path.join(root_study_dir,scan,scan_modal,scan_each))
                        print(os.path.join(root_study_dir,scan,scan_modal,scan_each), "Removed")
                    # if D_files == "nii":
                    #     shutil.rmtree(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files))
                    #     print(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files), "Removed")
                    if D_files == "nrrd":
                        shutil.rmtree(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files))
                        print(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files), "Removed")

 

Now processing subject: PatientData_Processed
Now processing subject: RP_01
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1\RP_01_1_CT\nrrd Removed
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1\RP_01_1_PT\nrrd Removed
Now processing subject: RP_02
Now processing subject: RP_03
Now processing subject: RP_04
Now processing subject: RP_05
Now processing subject: RP_06
Now processing subject: RP_08
Now processing subject: RP_09


In [17]:
# Rename PET CT folders
scan_dir = os.listdir(root_study_dir)
scan_dir.sort()
# i = 0
for scan in scan_dir:
    if "PatientData_Processed" not in scan:
        print("Now processing subject: " + str(scan))

        scan_session = os.listdir(os.path.join(root_study_dir,scan))
        scan_session.sort()
        for scan_modal in scan_session:
            if "_Rt" not in scan_modal:
                print("Now processing scan: " + str(scan_modal))
                scan_type = os.listdir(os.path.join(root_study_dir,scan,scan_modal))
                scan_type.sort() 
                for scan_each in scan_type:
                    print("New processing modal: ", scan_each)
                    dicom_files = os.listdir(os.path.join(root_study_dir,scan,scan_modal,scan_each))
                    dicom_files.sort()
                    for D_files in dicom_files:
                        if 'DICOM' in D_files:
                            files = os.listdir(os.path.join(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files)))
                            files.sort()
                            for file in files:
                                ds = pydicom.read_file(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files,file))
                                if ds.Modality == 'RWV':
                                    os.remove(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files,file))
                            files = os.listdir(os.path.join(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files)))
                            files.sort()
                            ds = pydicom.read_file(os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files,files[0]))
                            # print(ds.Modality)
                            os.rename(os.path.join(root_study_dir,scan,scan_modal,scan_each), os.path.join(root_study_dir,scan,scan_modal,scan_modal + "_" +str(ds.Modality)))


Now processing subject: RP_01
Now processing scan: RP_01_1
New processing modal:  RP_01_1_CT
New processing modal:  RP_01_1_PT
Now processing scan: RP_01_2
New processing modal:  RP_01_2_CT
New processing modal:  RP_01_2_PT
Now processing subject: RP_02
Now processing scan: RP_02_1
New processing modal:  RP_02_1_CT
New processing modal:  RP_02_1_PT
Now processing scan: RP_02_2
New processing modal:  RP_02_2_CT
New processing modal:  RP_02_2_PT
Now processing subject: RP_03
Now processing scan: RP_03_1
New processing modal:  RP_03_1_CT
New processing modal:  RP_03_1_PT
Now processing scan: RP_03_2
New processing modal:  RP_03_2_CT
New processing modal:  RP_03_2_PT
Now processing subject: RP_04
Now processing scan: RP_04_1
New processing modal:  RP_04_1_CT
New processing modal:  RP_04_1_PT
Now processing scan: RP_04_2
New processing modal:  RP_04_2_CT
New processing modal:  RP_04_2_PT
Now processing subject: RP_05
Now processing scan: RP_05_1
New processing modal:  RP_05_1_CT
New process

In [19]:
scan_dir = os.listdir(root_study_dir)
scan_dir.sort()
# i = 0
for scan in scan_dir:
    if "PatientData_Processed" not in scan:
        print("Now processing subject: " + str(scan))

        scan_session = os.listdir(os.path.join(root_study_dir,scan))
        scan_session.sort()
        for scan_modal in scan_session:
            print("Now processing scan: " + str(scan_modal))
            scan_type = os.listdir(os.path.join(root_study_dir,scan,scan_modal))
            scan_type.sort() 
            for scan_each in scan_type:
                # print("New processing modal: ", scan_each)
                dicom_files = os.listdir(os.path.join(root_study_dir,scan,scan_modal,scan_each))
                dicom_files.sort()
                for D_files in dicom_files:
                    if 'DICOM' in D_files or 'secondary' in D_files:
                        if 'CT' in scan_each:
                            output_folder = os.path.join(root_study_dir,scan,scan_modal,scan_each,'nii')
                            ct_path = os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files)
                            os.mkdir(output_folder)
                        #     # converts directory of CT/PET dicom to nii
                            dicom2nifti.convert_directory(ct_path, output_folder)
                            print('written CT: ', output_folder, 'from ', ct_path)
                        elif 'PT' in scan_each:
                            output_folder = os.path.join(root_study_dir,scan,scan_modal,scan_each,'nii')
                            pt_path = os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files)
                            os.mkdir(output_folder)
                        #     # converts directory of CT/PET dicom to nii
                            dicom2nifti.convert_directory(pt_path, output_folder)
                            print('written PT: ', output_folder, 'from ', pt_path)
                        elif 'Rt' in scan_modal:
                            output_folder = os.path.join(root_study_dir,scan,scan_modal,scan_each,'nii')
                            os.mkdir(output_folder)
                            full_path = os.path.join(root_study_dir,scan,scan_modal,scan_each,D_files)
                            rt_file = sorted(glob.glob(full_path + '\\2*.dcm'))
                            dicom_file_path = sorted(glob.glob(os.path.join(root_study_dir,scan) + "\\" + '_'.join(scan_modal.split('_')[:3]) + '\\*CT\\DICOM' ))
                            # print("Using: " + str(dicom_file_path))
                            # print("RT File: ", rt_file)
                            # converts rt struct dicom file to nii but needs dicom folder
                            dcmrtstruct2nii(rt_file[0], dicom_file_path[0], output_folder)
                            print('written RT: ', output_folder)

Now processing subject: RP_01
Now processing scan: RP_01_1
written CT:  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1\RP_01_1_CT\nii from  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1\RP_01_1_CT\DICOM
written PT:  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1\RP_01_1_PT\nii from  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1\RP_01_1_PT\DICOM
Now processing scan: RP_01_1_Rt
Using: ['C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST\\RP_01\\RP_01_1\\RP_01_1_CT\\DICOM']
RT File:  ['C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST\\RP_01\\RP_01_1_Rt\\1\\secondary\\2.16.840.1.114362.1.11882625.22367024715.562774123.599.1445.dcm']
written RT:  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nii
Now processing scan: RP_01_2
written CT:  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_2\RP_01_2_CT\nii 

In [26]:
def generate_nrrd_header(ds,img3d):
    # type, dimensions, space = 'left-posterior-superior', sizes, space direction, encoding, endian, kinds, space origin
    z_max_pos = ds[0].ImagePositionPatient[2]
    # z_list = []
    for ds_ct in ds:
        # z_list.append(ds_ct.ImagePositionPatient[2])
        if ds_ct.ImagePositionPatient[2] < z_max_pos:
            z_max_pos = ds_ct.ImagePositionPatient[2]

    # print(min(z_list))
    image_start = np.asarray((ds[0].ImagePositionPatient[0], ds[0].ImagePositionPatient[1], z_max_pos))
    nrrd_header = OrderedDict()

    nrrd_header['type'] = 'short'  # this gets overridden by the img3d data type
    nrrd_header['dimension'] = 3
    nrrd_header['space'] = 'left-posterior-superior'

    nrrd_header['sizes'] = np.array(img3d.shape)
    nrrd_header['space directions'] = np.array([[ds[0].PixelSpacing[0], 0, 0],
                                                [0, ds[0].PixelSpacing[1], 0],
                                                [0, 0, ds[0].SliceThickness]])
    nrrd_header['encoding'] = 'gzip'
    nrrd_header['endian'] = 'little'
    nrrd_header['kinds'] = ('domain', 'domain', 'domain')
    nrrd_header['space origin'] = np.array(image_start)

    return nrrd_header

In [ ]:
# root_study_dir = 'D:\\Melanoma01\\FinalTrainingCohort_AIQ_Mel01_Corrected\\Final_Mel01_nii'
# output_root = 'D:\\Melanoma01\\FinalTrainingCohort_AIQ_Mel01_Corrected\\Final_Mel01_nrrd'

In [31]:
def process_nii_to_nrrd(method,root_study_dir,sub,session):
    if method == 'CT':
        nii_files = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*CT*\\nii\\*nii*"))
        dicom_dir = sorted(glob.glob(os.path.join(root_study_dir, sub,session) + '\\*CT*\\DICOM'))
        dicom_dir = list(filter(lambda dir: dir[0] != '.' and "nii" not in dir and "PT" not in dir, dicom_dir))
        find_output_folder = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*CT*"))
    elif method == 'PT':
        nii_files = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*PT*\\nii\\*nii*"))
        dicom_dir = sorted(glob.glob(os.path.join(root_study_dir, sub,session) + '\\*PT*\\DICOM'))
        dicom_dir = list(filter(lambda dir: dir[0] != '.' and "nii" not in dir and "CT" not in dir, dicom_dir))
        find_output_folder = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*PT*"))
    elif method == 'RT':
        nii_files = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*\\nii\\*nii*"))
        dicom_dir = sorted(glob.glob(os.path.join(root_study_dir, sub) + "\\" + '_'.join(session.split('_')[:3]) + '\\*CT*\\DICOM'))
        find_output_folder = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*"))
        
    print('nii files: ', nii_files)
    print('ref dicom: ', dicom_dir)
    output_folder = os.path.join(find_output_folder[0],'nrrd')
    os.mkdir(output_folder)
    dicom_file = [pydicom.dcmread(f) for f in sorted(glob.glob(dicom_dir[0] + '\\2*.dcm'))]
    for nii_file in nii_files:
            a = nib.load(nii_file)
            image3d = a.get_fdata()
            header_info = generate_nrrd_header(dicom_file, image3d)
            output_file_name = os.path.join(output_folder, os.path.basename(nii_file).split('.')[0] + '.nrrd')
            nrrd.write(output_file_name, image3d, header_info)
            print('wrote: ', output_file_name)

In [32]:
subjects_scans = [os.path.basename(f) for f in sorted(glob.glob(root_study_dir + '\\RP*'))]
for sub in subjects_scans:
    print('subject: ', sub)
    sessions = os.listdir(os.path.join(root_study_dir, sub))
    sessions.sort()
    for session in sessions:
        print('Session: ', session)
        if 'Rt' not in session:
            modals = os.listdir(os.path.join(root_study_dir, sub,session))
            modals.sort()
            for modal in modals:
                if "CT" in modal:
                    method = 'CT'                    
                    process_nii_to_nrrd(method,root_study_dir,sub,session)
                elif "PT" in modal:
                    method = 'PT'
                    process_nii_to_nrrd(method,root_study_dir, sub,session)
        else:
            method = 'RT'
            process_nii_to_nrrd(method,root_study_dir, sub,session)

subject:  RP_01
Session:  RP_01_1
nii files:  ['C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST\\RP_01\\RP_01_1\\RP_01_1_CT\\nii\\4_ct_soft_tissue_whole_body.nii.gz']
ref dicom:  ['C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST\\RP_01\\RP_01_1\\RP_01_1_CT\\DICOM']
wrote:  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1\RP_01_1_CT\nrrd\4_ct_soft_tissue_whole_body.nrrd
nii files:  ['C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST\\RP_01\\RP_01_1\\RP_01_1_PT\\nii\\55447_corrected_for_suv_calculation.nii.gz']
ref dicom:  ['C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST\\RP_01\\RP_01_1\\RP_01_1_PT\\DICOM']
wrote:  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1\RP_01_1_PT\nrrd\55447_corrected_for_suv_calculation.nrrd
Session:  RP_01_1_Rt
nii files:  ['C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST\\RP_01\\RP_01_1_Rt\\1\\nii\\image.nii.gz', 'C:\\Users

In [ ]:
# combine rt nrrd files
import nrrd
import os
import numpy as np
import glob
from collections import OrderedDict
import pandas as pd
import shutil
import numpy as np
import napari
from skimage import data, filters  # Just to generate some test data (3D blobs).

In [ ]:
def myLabelStack(label_list):
    """given a list of str types, read in nrrd files and convert into one image array."""
    img3 = 0
    header = OrderedDict()
    for lab in label_list:
        mask = nrrd.read(lab)
        img3 = img3 + mask[0]

    header = mask[1]
    nrrd_img3 = (img3, header)
    return nrrd_img3

In [ ]:
def myNormaliseLabelImg3d(img3):
    """ given an 3D ndarry, normalise the data values and return. """
    img3_normalised = (img3 - np.min(img3))/(np.max(img3) - np.min(img3))
    return img3_normalised

In [ ]:
def myNormalisedLabelStack(label_list):
    """ given a list of str types, read in nrrd files, normalise values, and convert into one image array.
    returns: (tuple) image stack containing ndarry data and header information. """
    # mask = nrrd.read(label_list[0])
    # img3 = np.zeros(shape = mask[0].shape)
    img3 = 0
    header = OrderedDict()
    for lab in label_list:
        mask = nrrd.read(lab)
        img3 = img3 + mask[0]

        # print(mask[1])

    header = mask[1] # assume all file headers are the same and can just take the last one.
    overlap_index = np.where(img3>255)
    img3[overlap_index] = 255
    img3_stack = (myNormaliseLabelImg3d(img3), header)
    # img3_stack = (img3, header)

    return img3_stack

In [91]:
# delete lesion data
# for all patients:
# convert individual lesions into lesion stacks
# normalise mask into binary mask
# convert labels of lesions into specific numbers
root_dir = 'C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST'
subjects_scans = [os.path.basename(f) for f in sorted(glob.glob(root_dir + '\\RP*'))]
subjects_scans.sort()
# subjects = sorted(glob.glob(root_dir+'PS069*')) #'PS001LB'
# subject_id = [os.path.basename(f) for f in subjects]

for s in subjects_scans:
    if '.' not in s:
        print('processing: ', s)
        # output_dir = s + ''
        list_of_all_mask = sorted(glob.glob(os.path.join(root_dir,s) + '\\*Rt*\\*\\nrrd\\*lesion*.nrrd'))
        if len(list_of_all_mask) > 0:
            os.remove(list_of_all_mask[0])
            print(list_of_all_mask, "deleted")

processing:  RP_01
processing:  RP_02
processing:  RP_03
processing:  RP_04
processing:  RP_05
processing:  RP_06
processing:  RP_08
processing:  RP_09


In [92]:
# for all patients:
# convert individual lesions into lesion stacks
# normalise mask into binary mask
# convert labels of lesions into specific numbers
root_dir = 'C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST'
# subjects_scans = [os.path.basename(f) for f in sorted(glob.glob(root_dir + '\\RP*\\*Rt*'))]
subjects_scans = [f for f in sorted(glob.glob(root_dir + '\\RP*\\*Rt*'))]
subjects_scans.sort()
# subjects = sorted(glob.glob(root_dir+'PS069*')) #'PS001LB'
# subject_id = [os.path.basename(f) for f in subjects]

for s in subjects_scans:
    if '.' not in s:
        print('processing: ', s)
        # output_dir = s + ''
        list_of_all_mask = sorted(glob.glob(s + '\\*\\nrrd\\*mask*.nrrd'))
        lesion_mask = []
        for mask in list_of_all_mask:
            if 'Ref' not in mask and 'Burden' not in mask:
                lesion_mask.append(mask)
                # print(mask)
                
        lesion_mask_count = len(lesion_mask)
        print('mask total no.: ', lesion_mask_count)
        
        if lesion_mask_count != 0:
            img3d = myNormalisedLabelStack(lesion_mask)
            find_nrrd_dir = sorted(glob.glob(s + '\\*\\nrrd'))
            filename = os.path.join(find_nrrd_dir[0], 'lesionData.nrrd') 
            nrrd.write(filename, img3d[0], img3d[1])  # str, data, header
            print('written combined lesion mask: ', filename)
        
        # # total_count = 0
        # # for file in lesion_mask:
        # #     mask = nrrd.read(file)
        # #     lesion_index = np.where(mask[0] > 0)
        # #     print(len(lesion_index[2]))
        # #     total_count += len(lesion_index[2])


processing:  C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_19-1.nrrd
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_20-1.nrrd
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_25-1.nrrd
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_25-2.nrrd
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_25-3.nrrd
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_26-1.nrrd
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_30-1.nrrd
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_30-2.nrrd
C:\Users\julie\OneDrive\Documents\AIQ\mip_test\TEST_RETEST\RP_01\RP_01_1_Rt\1\nrrd\mask_30-3.nrrd
C:\Users\julie\OneDrive\Docum

In [93]:
source_folder = "C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST"
destination = "C:\\Users\\julie\\OneDrive\\Documents\\AIQ\\mip_test\\TEST_RETEST\\PatientData_Processed"

In [97]:
subjects_scans = [os.path.basename(f) for f in sorted(glob.glob(root_study_dir + '\\RP*'))]
for sub in subjects_scans:
    print('subject: ', sub)
    new_sub_dir = os.path.join(destination,sub)
    # print(new_sub_dir)
    os.makedirs(new_sub_dir)
    sessions = os.listdir(os.path.join(root_study_dir, sub))
    sessions.sort()
    for session in sessions:
        print('Session: ', session)
        if 'Rt' not in session:
            new_scan_dir = os.path.join(destination, sub, '_'.join(session.split('_')[:2]) + "_scan0" + session.split('_')[-1])
            # print(new_scan_dir)
            os.makedirs(new_scan_dir)
            pt_nrrd = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*PT*\\nrrd\\*.nrrd"))
            ct_nrrd = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*CT*\\nrrd\\*.nrrd"))
            shutil.copyfile(pt_nrrd[0],os.path.join(new_scan_dir, new_scan_dir.split('\\')[-1] + '_petData.nrrd'))
            shutil.copyfile(ct_nrrd[0],os.path.join(new_scan_dir, new_scan_dir.split('\\')[-1] + '_ctData.nrrd'))
        else:
            scan_dir = os.path.join(destination, sub, '_'.join(session.split('_')[:2]) + "_scan0" + session.split('_')[-2])
            # print(scan_dir)
            rt_nrrd = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + session + "\\*\\nrrd\\*lesion*.nrrd"))
            shutil.copyfile(rt_nrrd[0],os.path.join(scan_dir, scan_dir.split('\\')[-1] + '_lesionData.nrrd'))


subject:  RP_01
Session:  RP_01_1
Session:  RP_01_1_Rt
Session:  RP_01_2
Session:  RP_01_2_Rt
subject:  RP_02
Session:  RP_02_1
Session:  RP_02_1_Rt
Session:  RP_02_2
Session:  RP_02_2_Rt
subject:  RP_03
Session:  RP_03_1
Session:  RP_03_1_Rt
Session:  RP_03_2
Session:  RP_03_2_Rt
subject:  RP_04
Session:  RP_04_1
Session:  RP_04_1_Rt
Session:  RP_04_2
Session:  RP_04_2_Rt
subject:  RP_05
Session:  RP_05_1
Session:  RP_05_1_Rt
Session:  RP_05_2
Session:  RP_05_2_Rt
subject:  RP_06
Session:  RP_06_1
Session:  RP_06_1_Rt
Session:  RP_06_2
Session:  RP_06_2_Rt
subject:  RP_08
Session:  RP_08_1
Session:  RP_08_1_Rt
Session:  RP_08_2
Session:  RP_08_2_Rt
subject:  RP_09
Session:  RP_09_1
Session:  RP_09_1_Rt
Session:  RP_09_2
Session:  RP_09_2_Rt
